In [27]:
#file1=['../tmp/kgtk_roget_synonyms.tsv', '../tmp/kgtk_roget_antonyms.tsv']
#file1=['../tmp/kgtk_conceptnet.tsv']

files=['../tmp/kgtk_wordnet_extended.tsv', 
       '../tmp/kgtk_wikidata.tsv', 
       '../tmp/kgtk_conceptnet.tsv',
       '../tmp/kgtk_roget.tsv']

In [28]:
from tqdm import tqdm
import json

In [29]:
dim_file='../consolidation/dimensions.json'

In [30]:
with open(dim_file, 'r') as f:
    dim_mapping=json.load(f)

In [31]:
dimensions=set(dim_mapping.values())

In [32]:
len(dimensions)

12

In [33]:
dimensions

{'creation',
 'desire',
 'distinctness',
 'lexical',
 'part-whole',
 'quality',
 'rel-other',
 'similarity',
 'spatial',
 'taxonomic',
 'temporal',
 'utility'}

In [34]:
def get_all_labels(l):
    if '|' not in l:
        return [l]
    else:
        return [l2 for l2 in l.split('|') if l2!='']

In [35]:
def get_triples(a_file, abstract_rel=False, dim_mapping={}, dim_filter=None):
    triples=set()
    with open(a_file, 'r') as f1:
        header=next(f1)
        for line in tqdm(f1, total=3500000):
            data=line.split('\t')
            rel_label=data[1]
            if abstract_rel:
                if rel_label not in dim_mapping.keys(): continue
                rel_label=dim_mapping[rel_label]
                if dim_filter:
                    if rel_label!=dim_filter: continue
            for n1_label in get_all_labels(data[3]):
                for n2_label in get_all_labels(data[4]):
                    if n1_label=='' or n2_label=='': print(data)
                    triple=(n1_label, rel_label, n2_label)
                    triples.add(triple)
    return triples

## Compute overlaps between each source pair

In [36]:
def file_to_source(f):
    return f.split('/')[-1].replace('kgtk_', '').replace('.tsv', '')

In [37]:
for file1 in files:
    for file2 in files:
        if file1<file2:

            f1_triples=get_triples(file1)
            f2_triples=get_triples(file2)
            both_triples=set(f1_triples) & set(f2_triples)

            only_src2=len(set(f2_triples)-set(f1_triples))
            pct_only_src2=only_src2/(only_src2+len(both_triples))

            only_src1=len(set(f1_triples)-set(f2_triples))
            pct_only_src1=only_src1/(only_src1+len(both_triples))

            f1=file_to_source(file1)
            f2=file_to_source(file2)
            
            union=set(f1_triples)|set(f2_triples)

            print(f1, '-', f2, 'BOTH:', len(both_triples), 'ONLY S1:', only_src1, 'ONLY S2:', only_src2, 'UNION:', len(union), 'JACCARD:', len(both_triples)*100.0/len(union))


  3%|▎         | 101771/3500000 [00:00<00:05, 604121.72it/s][A

 14%|█▎        | 474235/3500000 [00:00<00:05, 507717.99it/s]

  0%|          | 4225/3500000 [00:00<01:46, 32977.09it/s]

wikidata - wordnet_extended BOTH: 1697 ONLY S1: 98162 ONLY S2: 781978 UNION: 881837 JACCARD: 0.1924391922770308



 98%|█████████▊| 3423004/3500000 [00:05<00:00, 675546.46it/s]

 14%|█▎        | 474235/3500000 [00:00<00:05, 513973.79it/s]

  0%|          | 0/3500000 [00:00<?, ?it/s]


conceptnet - wordnet_extended BOTH: 86006 ONLY S1: 3237315 ONLY S2: 697669 UNION: 4020990 JACCARD: 2.1389259858890473


 98%|█████████▊| 3423004/3500000 [00:05<00:00, 624298.42it/s]

  3%|▎         | 101771/3500000 [00:00<00:05, 636267.51it/s][A

  0%|          | 0/3500000 [00:00<?, ?it/s]


...

conceptnet - roget BOTH: 57635 ONLY S1: 3265686 ONLY S2: 1345826 UNION: 4669147 JACCARD: 1.2343796415062538



 40%|████      | 1403955/3500000 [00:02<00:03, 547694.85it/s]

 14%|█▎        | 474235/3500000 [00:00<00:05, 528070.91it/s]

  0%|          | 2674/3500000 [00:00<02:45, 21154.75it/s]

roget - wordnet_extended BOTH: 75025 ONLY S1: 1328436 ONLY S2: 708650 UNION: 2112111 JACCARD: 3.5521333869290013



 40%|████      | 1403955/3500000 [00:02<00:03, 596246.15it/s]

  3%|▎         | 101771/3500000 [00:00<00:05, 652859.56it/s][A


roget - wikidata BOTH: 299 ONLY S1: 1403162 ONLY S2: 99560 UNION: 1503021 JACCARD: 0.019893268290995268


## Compute mappings (with dimensions instead of relations)

In [38]:
for file1 in files:
    for file2 in files:
        if file1<file2:

            f1_triples=get_triples(file1, True, dim_mapping)
            f2_triples=get_triples(file2, True, dim_mapping)
            both_triples=set(f1_triples) & set(f2_triples)

            only_src2=len(set(f2_triples)-set(f1_triples))
            pct_only_src2=only_src2/(only_src2+len(both_triples))

            only_src1=len(set(f1_triples)-set(f2_triples))
            pct_only_src1=only_src1/(only_src1+len(both_triples))

            f1=file_to_source(file1)
            f2=file_to_source(file2)

            union=set(f1_triples)|set(f2_triples)

            print(f1, '-', f2, 'BOTH:', len(both_triples), 'ONLY S1:', only_src1, 'ONLY S2:', only_src2, 'UNION:', len(union), 'JACCARD:', len(both_triples)*100.0/len(union))
            
#             for b in both_triples:
#                 print(b)
#                 input('c?')


  3%|▎         | 101771/3500000 [00:00<00:12, 272880.33it/s]A

 14%|█▎        | 474235/3500000 [00:00<00:04, 606638.87it/s]

  0%|          | 5610/3500000 [00:00<01:02, 56098.85it/s]

wikidata - wordnet_extended BOTH: 1704 ONLY S1: 98091 ONLY S2: 574967 UNION: 674762 JACCARD: 0.25253348588094765



 98%|█████████▊| 3423004/3500000 [00:05<00:00, 594553.85it/s]

 14%|█▎        | 474235/3500000 [00:00<00:04, 607007.65it/s]

  0%|          | 0/3500000 [00:00<?, ?it/s]

conceptnet - wordnet_extended BOTH: 97946 ONLY S1: 3185302 ONLY S2: 478725 UNION: 3761973 JACCARD: 2.6035806211262016



 98%|█████████▊| 3423004/3500000 [00:05<00:00, 579976.22it/s]

  3%|▎         | 101771/3500000 [00:00<00:05, 589546.12it/s][A

  0%|          | 0/3500000 [00:00<?, ?it/s]

...

conceptnet - roget BOTH: 73992 ONLY S1: 3209256 ONLY S2: 1329469 UNION: 4612717 JACCARD: 1.6040871356296083



 40%|████      | 1403955/3500000 [00:02<00:04, 515279.41it/s]

 14%|█▎        | 474235/3500000 [00:00<00:05, 569163.47it/s]

  0%|          | 2673/3500000 [00:00<02:37, 22230.50it/s]

roget - wordnet_extended BOTH: 75025 ONLY S1: 1328436 ONLY S2: 501646 UNION: 1905107 JACCARD: 3.9380990149109736



 40%|████      | 1403955/3500000 [00:02<00:03, 528366.93it/s]

  3%|▎         | 101771/3500000 [00:00<00:06, 564641.85it/s][A


roget - wikidata BOTH: 333 ONLY S1: 1403128 ONLY S2: 99462 UNION: 1502923 JACCARD: 0.022156823736146164


### Compute mappings per dimension

In [39]:
for file1 in files:
    for file2 in files:
        if file1<file2:

            for d in dimensions:

                f1_triples=get_triples(file1, True, dim_mapping, d)
                f2_triples=get_triples(file2, True, dim_mapping, d)
                both_triples=set(f1_triples) & set(f2_triples)
            
                try:

                    only_src2=len(set(f2_triples)-set(f1_triples))
                    pct_only_src2=only_src2/(only_src2+len(both_triples))

                    only_src1=len(set(f1_triples)-set(f2_triples))
                    pct_only_src1=only_src1/(only_src1+len(both_triples))

                    f1=file_to_source(file1)
                    f2=file_to_source(file2)

                    union=set(f1_triples)|set(f2_triples)

                    print(f1, '-', f2, d, 'BOTH:', len(both_triples), 'ONLY S1:', only_src1, 'ONLY S2:', only_src2, 'UNION:', len(union), 'JACCARD:', len(both_triples)*100.0/len(union))
                except:
                    continue


  3%|▎         | 101771/3500000 [00:00<00:04, 846756.68it/s][A

 14%|█▎        | 474235/3500000 [00:00<00:02, 1148127.29it/s]

  3%|▎         | 101771/3500000 [00:00<00:08, 390015.68it/s]A

 14%|█▎        | 474235/3500000 [00:00<00:03, 797276.05it/s]

  3%|▎         | 101771/3500000 [00:00<00:04, 709951.79it/s][A

  0%|          | 0/3500000 [00:00<?, ?it/s]

wikidata - wordnet_extended taxonomic BOTH: 1533 ONLY S1: 69884 ONLY S2: 318236 UNION: 389653 JACCARD: 0.39342697220347334



 14%|█▎        | 474235/3500000 [00:00<00:03, 961443.27it/s]

  3%|▎         | 101771/3500000 [00:00<00:04, 804253.81it/s][A

  0%|          | 0/3500000 [00:00<?, ?it/s]

wikidata - wordnet_extended part-whole BOTH: 82 ONLY S1: 8121 ONLY S2: 100839 UNION: 109042 JACCARD: 0.07520038150437446



 14%|█▎        | 474235/3500000 [00:00<00:02, 1151881.21it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1030613.61it/s]

 14%|█▎        | 474235/3500000 [00:00<00:02, 1151089.30it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1083889.67it/s]

 14%|█▎        | 474235/3500000 [00:00<00:02, 1152532.63it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1095503.39it/s]

 14%|█▎        | 474235/3500000 [00:00<00:02, 1082565.17it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1050007.53it/s]

 14%|█▎        | 474235/3500000 [00:00<00:03, 948598.91it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 914353.28it/s][A

  0%|          | 0/3500000 [00:00<?, ?it/s]

...

roget - wordnet_extended distinctness BOTH: 3300 ONLY S1: 377550 ONLY S2: 253 UNION: 381103 JACCARD: 0.8659076417661367



 40%|████      | 1403955/3500000 [00:01<00:01, 1201267.91it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 948627.17it/s][A

 40%|████      | 1403955/3500000 [00:01<00:01, 1263227.39it/s]

  3%|▎         | 101771/3500000 [00:00<00:07, 468131.82it/s][A

 40%|████      | 1403955/3500000 [00:01<00:01, 1215809.51it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 927070.56it/s][A

 40%|████      | 1403955/3500000 [00:01<00:01, 1226699.02it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1036310.85it/s]

 40%|████      | 1403955/3500000 [00:01<00:01, 1213223.93it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1057449.47it/s]

 40%|████      | 1403955/3500000 [00:01<00:01, 1198398.54it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 979529.79it/s] 

 40%|████      | 1403955/3500000 [00:01<00:01, 1229320.34it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1023729.90it/s]

 40%|████      | 1403955/3500000 [00:01<00:02, 719216.44it/s]

  3%|▎         | 101771/3500000 [00:00<00:03

roget - wikidata similarity BOTH: 127 ONLY S1: 1022484 ONLY S2: 1198 UNION: 1023809 JACCARD: 0.012404657509359656



 40%|████      | 1403955/3500000 [00:01<00:01, 1212306.53it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 942935.72it/s][A

 40%|████      | 1403955/3500000 [00:01<00:01, 1247377.89it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1045220.17it/s]

 40%|████      | 1403955/3500000 [00:01<00:01, 1204606.01it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 1029477.69it/s]

 40%|████      | 1403955/3500000 [00:01<00:02, 867835.57it/s]

  3%|▎         | 101771/3500000 [00:00<00:03, 997587.03it/s] 


roget - wikidata distinctness BOTH: 206 ONLY S1: 380644 ONLY S2: 6515 UNION: 387365 JACCARD: 0.053179817484801155


## Future work

* Incorporate node resolution for higher recall for overlaps